# Descripción

Contamos con unos datasets que corresponden a un **listado de inspecciones de sanidad en locales** (Restaurantes, supermercados, etc), junto con su respectivo riesgo para la salud. Contamos con otro dataset que nos muestra una **descripción de dicho riesgo**.

**El objetivo es cargar esos datasets bajo unas especificaciones concretas y manipularlos acorde a las instrucciones de cada ejercicio.**

Todas las operaciones necesarias están descritas en los ejercicios, aunque se valorará tareas extras por propia iniciativa del alumno. También se valorará el uso del API de DataFrame.

**La entrega será un fichero zip con tu nombre y apellidos** que contendrá:
- Este fichero `ipynb`, con las correspondientes soluciones.
- Imagen del plan de ejecución del ejercicio 4.
- Descripción del plan de ejecución que podrá estar embebido en el fichero `ipynb`.

El ejercicio 10 tienes dos opciones, puedes optar por el `10a` que es continuación de este proyecto o por el `10b`

# Descargar Datasets

In [0]:
%sh 
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/food_inspections_lite.csv' --output-dir /databricks/driver
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/risk_description.csv'  --output-dir /databricks/driver

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7361k  100 7361k    0     0  5792k      0  0:00:01  0:00:01 --:--:-- 5796k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   176  100   176    0     0    566      0 --:--:-- --:--:-- --:--:--   565


In [0]:
dbutils.fs.cp('file:/databricks/driver/food_inspections_lite.csv','dbfs:/dataset/food_inspections_lite.csv')
dbutils.fs.cp('file:/databricks/driver/risk_description.csv','dbfs:/dataset/risk_description.csv')

True

In [0]:
dbutils.fs.ls('/dataset/')

[FileInfo(path='dbfs:/dataset/bank.csv', name='bank.csv', size=461474, modificationTime=1746871963000),
 FileInfo(path='dbfs:/dataset/books/', name='books/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dataset/characters.csv', name='characters.csv', size=5462, modificationTime=1747469386000),
 FileInfo(path='dbfs:/dataset/food_inspections_lite.csv', name='food_inspections_lite.csv', size=7538077, modificationTime=1747849257000),
 FileInfo(path='dbfs:/dataset/frankenstein.txt', name='frankenstein.txt', size=421623, modificationTime=1747469385000),
 FileInfo(path='dbfs:/dataset/melb_data.csv', name='melb_data.csv', size=2091239, modificationTime=1746871964000),
 FileInfo(path='dbfs:/dataset/netflix_titles.csv', name='netflix_titles.csv', size=2401846, modificationTime=1746805933000),
 FileInfo(path='dbfs:/dataset/partitioned/', name='partitioned/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dataset/planets.csv', name='planets.csv', size=4289, modificationTime=1746871963000

In [0]:
KAFKA_BOOSTRAP_SERVER="35.227.18.205:9094"

In [0]:
checkpoint_path = "/tmp/project_spark/_checkpoint"

In [0]:
spark.conf.set("spark.sql.streaming.checkpointLocation", checkpoint_path)
spark.conf.get("spark.sql.streaming.checkpointLocation")

'/tmp/project_spark/_checkpoint'

# Ejercicio 1
---

1. **Crea dos dataframes, uno a partir del fichero `food_inspections_lite.csv` y otro a partir de `risk_description.csv`**
2. **Convierte esos dos dataframes a tablas delta**


In [0]:
dbutils.fs.head("/dataset/food_inspections_lite.csv")

[Truncated to first 65536 bytes]


'Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location\n2373596,COUNTY BBQ,COUNTY BBQ,2732781,Restaurant,Risk 3 (Low),1352 W TAYLOR ST ,CHICAGO,IL,60607,06/11/2020,License,Fail,,41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"\n2373587,KIDZ CREATIVE CORNER,KIDZ CREATIVE CORNER,2555611,Daycare Above and Under 2 Years,Risk 1 (High),4259 N WESTERN AVE ,CHICAGO,IL,60618,06/11/2020,License Re-Inspection,Pass,,41.95944802136064,-87.68848155320953,"(-87.68848155320953, 41.95944802136064)"\n2373578,DSD DELI,DSD DELI,2626186,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE ,CHICAGO,IL,60630,06/11/2020,License Re-Inspection,Pass,,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"\n2373575,DSD DELI,DSD DELI,2626178,DELI/GROCERY,Risk 1 (High),5205 N MILWAUKEE ,CHICAGO,IL,60630,06/11/2020,License Re-Inspection,Pass,,41.97484853157327,-87.76

In [0]:
dbutils.fs.head("/dataset/risk_description.csv")

'risk_id,description\n1,Este riesgo significar la clausura inmediata del local\n2,Este riesgo está cerca de convertirse en la clausura del local\n3,Necesita una mejora importante\n'

In [0]:
# Part 1 -> create dataframes
food_inspections_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/food_inspections_lite.csv")

risk_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/risk_description.csv")


In [0]:
# Part 2 -> Create delta tables

# Ejercicio 2
**Obtén el número de inspecciones distintas con Riesgo alto `Risk 1 (High)`**

---



In [0]:
from pyspark.sql.functions import col

# Se asume que cada inspección es distinta
food_inspections_df \
  .filter(food_inspections_df.Risk == "Risk 1 (High)") \
  .groupBy(col("Risk")) \
  .count() \
  .select(
    col("Risk"),
    col("count").alias("number")
  ).display()


Risk,number
Risk 1 (High),7215



# Ejercicio 3
**A partir de los dataframes cargados anteriormente, obtén una tabla con las siguientes columnas:<br>**
1. `DBA Name`
2. `Facility Type`
3. `Risk`
4. `Risk description`

---

In [0]:
# Esto nos sirve para ver lo que nos encontramos en la columna Risk y tenerlo en cuenta en el siguiente.

food_inspections_df \
  .select(food_inspections_df.Risk) \
  .distinct().display()

Risk
null
Risk 1 (High)
All
Risk 2 (Medium)
Risk 3 (Low)


In [0]:
# Tenemos que hacer un join de los dos dataframes, pero necesitamos formatear el nivel de riesgo para encajar con el segundo
# Se pueden usar funciones que ya existen pero por mostrar otro elemento que hemos visto en clase, vamos a crear una udf

from pyspark.sql.functions import col, udf

@udf()
def get_risk_level(risk):
  if risk is not None:
    return risk.split(" ")[1] # segunda "palabra" [Risk, 1, (High)]

# No se menciona nada de ordenar, nos lo ahorramos y evitamos shuffle innecesario

food_inspections_df \
  .na.drop(subset=["Risk"]) \
  .filter(food_inspections_df.Risk != "All") \
  .withColumn("risk_level", get_risk_level(col("Risk"))) \
  .join(risk_df, col("risk_level") == risk_df.risk_id) \
  .select(
    food_inspections_df["DBA Name"],
    food_inspections_df["Facility Type"],
    col("Risk"),
    risk_df["description"].alias("Risk description")
  ).display()

DBA Name,Facility Type,Risk,Risk description
COUNTY BBQ,Restaurant,Risk 3 (Low),Necesita una mejora importante
KIDZ CREATIVE CORNER,Daycare Above and Under 2 Years,Risk 1 (High),Este riesgo significar la clausura inmediata del local
DSD DELI,DELI/GROCERY,Risk 3 (Low),Necesita una mejora importante
DSD DELI,DELI/GROCERY,Risk 1 (High),Este riesgo significar la clausura inmediata del local
COUNTY BBQ,Restaurant,Risk 1 (High),Este riesgo significar la clausura inmediata del local
NEW SEOUL,null,Risk 3 (Low),Necesita una mejora importante
"CAFE UTJEHA, INC.",Restaurant,Risk 2 (Medium),Este riesgo está cerca de convertirse en la clausura del local
MARATHON GAS,Grocery Store,Risk 3 (Low),Necesita una mejora importante
HALSTED BOWL,Restaurant,Risk 2 (Medium),Este riesgo está cerca de convertirse en la clausura del local
HAROLD'S CHICKEN,Restaurant,Risk 2 (Medium),Este riesgo está cerca de convertirse en la clausura del local


# Ejercicio 4
**Accede a la Spark UI para ver el plan de ejecución. Describe cada una de las piezas/cajas que componen el plan de ejecución (Una descripción breve de una línea por caja será suficiente).**<br><br>**Recordad hacer un pantallazo del plan de ejecución analizado**

---



# Ejercicio 5
**1. Obtén el número de inspecciones para cada local (columna `DBA Name`) y su resultado (columna `Results`).**<br><br>
**2. Obtén las dos locales (`DBA Name`) que más inspecciones han tenido por cada uno de los resultados**<br><br>
**3. Guarda los resultados en una nueva tabla Delta llamada `inspections_results`**

---

In [0]:
# Esto nos sirve para ver lo que nos encontramos en la columna Risk y tenerlo en cuenta en el siguiente.

food_inspections_df \
  .select(food_inspections_df.Results) \
  .distinct().display()

Results
Not Ready
Fail
No Entry
Business Not Located
Pass w/ Conditions
Out of Business
Pass


In [0]:
from pyspark.sql.functions import col

# Primer paso, obtener el número de inspecciones por local y resultado

inspections_grouped = food_inspections_df \
  .groupBy(
    food_inspections_df["DBA Name"],
    food_inspections_df["Results"]
  ) \
  .count() \
  .select(
    food_inspections_df["DBA Name"],
    col("count").alias("Num inspections"),
    food_inspections_df["Results"]
  )

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, desc

# Segundo paso, mediante window partitioning obtenemos el top dos de locales por result

by_result = Window.partitionBy("Results").orderBy(desc("Num inspections"))

inspections_grouped \
  .withColumn("inspection_row_num", row_number().over(by_result)) \
  .filter(col("inspection_row_num") <= 2) \
  .select(
    inspections_grouped["DBA Name"],
    inspections_grouped["Num inspections"],
    inspections_grouped["Results"],
    col("inspection_row_num")
  ).display()

DBA Name,Num inspections,Results,inspection_row_num
RICE THAI CAFE,1,Business Not Located,1
CHINA STATION,1,Business Not Located,2
SUBWAY,10,Fail,1
DUNKIN DONUTS,5,Fail,2
LANS,5,No Entry,1
LA PENA RESTAURANTE,4,No Entry,2
PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,5,Not Ready,1
SUBWAY,4,Not Ready,2
SUBWAY,16,Out of Business,1
DUNKIN DONUTS,10,Out of Business,2


In [0]:
# Tercer paso, transformarlo en la tabla delta

# Ejercicio 6
1. **Actualiza la tabla delta creada en el ejercicio anterios, con el valor `DBA_Name = "error"`**
2. **Restaura la tabla a su estado original**

---



# Ejercicio 7

**Crea una aplicación son Structured Streaming que lea los datos del topic de Kafka `inspections`. La url del servidor Kafka está definida al comienzo de este notebook**

**Los datos procedentes de este topic son exactamente los mismos que estamos analizando durante todo este notebook, `Food Inspections`, así que el esquema es el mismo**

# Ejercicio 8
**En base a la fuente de datos del ejercicio anterior, obtén cada 5 segundos el número de inspecciones por `Facility Type`**

# Ejercicio 9
**En base a la fuente de datos del ejercicio 7, obtén cada 5 segundos el número de inspecciones por `Results` de los últimos 30 segundos**

# Ejercicio 10a
1. **Actualiza la columna `Results` de la tabla delta de food inspections creada en el ejercicio 1 con el valor `No result`**
2. **Ahora que tenemos la tabla delta corrompida con el valor `No result`, vamos a resolver el problema con los datos que nos llegan de Kafka, los cuales vamos a suponer como verdad absoluta, y por tanto tendremos que actualizar en tiempor real conforme vayan llegando elementos en Kafka**.<br>

---

Se aconseja para todos los streams anteriores ya que el de este ejercicio suele hacer un uso intensivo de los recursos

# Ejercicio 10b
Diseñar una solución de análisis en tiempo real utilizando Apache Spark en Databricks para consumir datos de vuelos transmitidos por Kafka, almacenarlos en una tabla Delta y visualizar la posición actual de los vuelos sobre un mapa.
- Los datos de los vuelos están en un topic llamado `flights`
- Guardar en una tabla delta todos los vuelos, pero sólo una entrada por código de vuelo, de manera que si recibimos actualizaciones de posiciones del vuelo, se actualizará el registro correspondiente. Esto debe suceder en tiempo real.

---

Para más información sobre los datos de entrada consultar [OpenSky Network](https://openskynetwork.github.io/opensky-api/rest.html#all-state-vectors). A continuación se muestra un pantallazo de la visualización que se pretende conseguir. Te en cuenta que esta visualización de mapa está disponible en Databricks, con lo que no necesitará importar ninguna librería externa

![Flight Map](https://raw.githubusercontent.com/masfworld/datahack_docker/ab487794745499248388b67cf574085c5d86746e/zeppelin/data/image.png)